In [ ]:
 !pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 68.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 KB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.6/99.6 KB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 KB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 KB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 KB 11

In [ ]:
!pip install geehydro

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for geehydro: filename=geehydro-0.2.0-py2.py3-none-any.whl size=10140 sha256=83971c42180a73d62f8e759a631c72478d0f0efb66c39f517bddb77a13a18b27
  Stored in directory: /root/.cache/pip/wheels/cc/f0/c8/f761318de15d20d46693d576710a46d56b7bd9ba15249800e7
Successfully built geehydro


In [ ]:
import ee, datetime
import pandas as pd
import numpy as np
import folium
import geehydro
import datetime as dt
import IPython.display


## Authentication and Intialisation

In [ ]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=Tb3UAueDaMuPEGiGjrB1pyEWbNA-RhcRYZUBvDVIflo&tc=35J-_Ewwb_24rWtfu8II6SDd5tCxe3_oqe6YKtnyEDs&cc=66q9ueh67lumbOP-PbTcG1Jvwwkw6jvfZ2b2J3Z8B1w

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh5mKfSRB3SJJdOyQSjMJUfj8KXIYAwJXIi07FgUQ21lIfQ-Y86EVi4

Successfully saved authorization token.


## Selecting the area of Interest

In [ ]:
lendiya = folium.Map(location=[23.244061109840693,77.47089584472357], zoom_start=10)


In [ ]:
lendiya

### Selecting the satellite and AOI


In [ ]:
sentinel = ee.ImageCollection("COPERNICUS/S2_SR").\
               filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).\
               filterDate("2015-01-01","2023-01-01")
lendiya_AOI = ee.Geometry.Polygon([[77.39023917078465, 23.27477380472318],
          [77.39034645914525, 23.274596401498602],
          [77.39034645914525, 23.273876930443446],
          [77.39060395121068, 23.273847363056678],
          [77.3905717647025, 23.27359111209644],
          [77.3909150874564, 23.273729093443976],
          [77.39105456232518, 23.273847363056678],
          [77.39120476603001, 23.273876930443446],
          [77.39134424089879, 23.27427116163988],
          [77.39111893534154, 23.275808652157437],
          [77.39099018930882, 23.27569038428615],
          [77.39077561258763, 23.27569038428615],
          [77.3904108321616, 23.275394714148526]])

sentinel_AOI = sentinel.filterBounds(lendiya_AOI)
               

In [ ]:
print('Total number of images :', sentinel_AOI.size().getInfo())

Total number of images : 166


We have 166 images for Lendiya Lake.
Now, we will create an iteration over all the images and find the different parameters 

## **NDCI Calculation**

In [ ]:
def addNDCI(image):
  ndci = image.normalizedDifference(['B5', 'B4']).rename('NDCI')
  return image.addBands(ndci)
with_ndci = sentinel_AOI.map(addNDCI)  

### Calculating the Average NDCI on every image in the image collection

In [ ]:
def meanNDCI(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['NDCI']),
                           geometry = lendiya_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('NDCI').getInfo()

In [ ]:
Images_ndci = with_ndci.select('NDCI').toList(with_ndci.size())
ndci_coll = []
for i in range(Images_ndci.length().getInfo()):
  image = ee.Image(Images_ndci.get(i-1))
  temp_ndci = meanNDCI(image)
  ndci_coll.append(temp_ndci)

## Getting Dates from the Image Collection

In [ ]:
dates = np.array(with_ndci.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

### Creating the Dataframe

In [ ]:
df = pd.DataFrame(ndci_coll, index = day, columns = ['Chlorophyll'])
df.index = pd.to_datetime(df.index, format="%Y/%m/%d")
df.sort_index(ascending = True, inplace = True)

In [ ]:
df

In [ ]:
df.to_csv('Chlorophyll.csv')

## **NDTI Calculation**

In [ ]:
def addNDTI(image):
  ndti = image.normalizedDifference(['B4', 'B3']).rename('NDTI')
  return image.addBands(ndti)
with_ndti = sentinel_AOI.map(addNDTI)

In [ ]:
def meanNDTI(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['NDTI']),
                           geometry = lendiya_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('NDTI').getInfo()

In [ ]:
Images_ndti = with_ndti.select('NDTI').toList(with_ndti.size())
ndti_coll = []
for i in range(Images_ndti.length().getInfo()):
  image = ee.Image(Images_ndti.get(i-1))
  temp_ndti = meanNDTI(image)
  ndti_coll.append(temp_ndti)

In [ ]:
dates = np.array(with_ndti.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

In [ ]:
df1 = pd.DataFrame(ndti_coll, index = day, columns = ['Turbidity'])
df1.index = pd.to_datetime(df1.index, format="%Y/%m/%d")
df1.sort_index(ascending = True, inplace = True)

In [ ]:
df1

,Turbidity
2018-12-18,-0.021874
2018-12-23,-0.123111
2018-12-28,-0.160459
2019-01-02,-0.121576
2019-01-07,-0.134198
...,...
2022-12-02,-0.027580
2022-12-07,-0.015449
2022-12-17,-0.016148
2022-12-22,-0.014442


In [ ]:
df1.to_csv('Turbidity.csv')

## **NDSI Calculation**

In [ ]:
def addNDSI(image):
  ndsi = image.normalizedDifference(['B11', 'B12']).rename('NDSI')
  return image.addBands(ndsi)
with_ndsi = sentinel_AOI.map(addNDSI)

In [ ]:
def meanNDSI(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['NDSI']),
                           geometry = lendiya_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('NDSI').getInfo()

In [ ]:
Images_ndsi = with_ndsi.select('NDSI').toList(with_ndsi.size())
ndsi_coll = []
for i in range(Images_ndsi.length().getInfo()):
  image = ee.Image(Images_ndsi.get(i-1))
  temp_ndsi = meanNDSI(image)
  ndsi_coll.append(temp_ndsi)

In [ ]:
dates = np.array(with_ndsi.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

In [ ]:
df2 = pd.DataFrame(ndsi_coll, index = day, columns = ['Salinty'])
df2.index = pd.to_datetime(df2.index, format="%Y/%m/%d")
df2.sort_index(ascending = True, inplace = True)

In [ ]:
df2

,Salinty
2018-12-18,0.060754
2018-12-23,0.215919
2018-12-28,0.162066
2019-01-02,0.172895
2019-01-07,0.133760
...,...
2022-12-02,0.062454
2022-12-07,0.040098
2022-12-17,0.048394
2022-12-22,0.046412


In [ ]:
df2.to_csv('Salinty.csv')

# **Dissloved Oxygen Calculation**

In [ ]:
def addDO(image):
  do = image.normalizedDifference(['B4', 'B5']).rename('DO')
  return image.addBands(do)
with_do = sentinel_AOI.map(addDO)

In [ ]:
def meanDO(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['DO']),
                           geometry = lendiya_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('DO').getInfo()

In [ ]:
Images_do = with_do.select('DO').toList(with_do.size())
do_coll = []
for i in range(Images_do.length().getInfo()):
  image = ee.Image(Images_do.get(i-1))
  temp_do = meanDO(image)
  do_coll.append(temp_do)

In [ ]:
dates = np.array(with_do.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

In [ ]:
df4 = pd.DataFrame(do_coll, index = day, columns = ['Dissolved Oxygen'])
df4.index = pd.to_datetime(df4.index, format="%Y/%m/%d")
df4.sort_index(ascending = True, inplace = True)

In [ ]:
df4

,Dissolved Oxygen
2018-12-18,-0.056721
2018-12-23,-0.240334
2018-12-28,-0.279906
2019-01-02,-0.256591
2019-01-07,-0.264053
...,...
2022-12-02,-0.086346
2022-12-07,-0.061451
2022-12-17,-0.077955
2022-12-22,-0.069670


In [ ]:
df4.to_csv('Dissolved Oxygen.csv')

# **pH**

In [ ]:
def addpH(image):
  ph = ee.Image(8.339).subtract(ee.Image(0.827).multiply(image.select('B1').divide(image.select('B8')))).rename('PH')
  return image.addBands(ph)
with_pH = sentinel_AOI.map(addpH)

In [ ]:
def meanpH(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['PH']),
                           geometry = lendiya_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('PH').getInfo()

In [ ]:
Images_ph = with_pH.select('PH').toList(with_pH.size())
ph_coll= []
for i in range(Images_ph.length().getInfo()):
  image = ee.Image(Images_ph.get(i-1))
  temp_ph = meanpH(image)
  ph_coll.append(temp_ph)

In [ ]:
dates = np.array(with_pH.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

In [ ]:
df6 = pd.DataFrame(ph_coll, index = day, columns = ['pH'])
df6.index = pd.to_datetime(df6.index, format="%Y/%m/%d")
df6.sort_index(ascending = True, inplace = True)

In [ ]:
df6

,pH
2018-12-18,7.662761
2018-12-23,7.820192
2018-12-28,7.873315
2019-01-02,7.701345
2019-01-07,7.922840
...,...
2022-12-02,7.633734
2022-12-07,7.669205
2022-12-17,7.689675
2022-12-22,7.629132


In [ ]:
df6.to_csv('pH.csv')

# **Temperature**


Selecting the Satellite



In [ ]:
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").\
              filterDate("2016-01-01","2022-01-01")
lendiya_AOI = ee.Geometry.Polygon([[77.39023917078465, 23.27477380472318],
          [77.39034645914525, 23.274596401498602],
          [77.39034645914525, 23.273876930443446],
          [77.39060395121068, 23.273847363056678],
          [77.3905717647025, 23.27359111209644],
          [77.3909150874564, 23.273729093443976],
          [77.39105456232518, 23.273847363056678],
          [77.39120476603001, 23.273876930443446],
          [77.39134424089879, 23.27427116163988],
          [77.39111893534154, 23.275808652157437],
          [77.39099018930882, 23.27569038428615],
          [77.39077561258763, 23.27569038428615],
          [77.3904108321616, 23.275394714148526]])

landsat_AOI = landsat.filterBounds(lendiya_AOI)

In [ ]:
print('Total number of images :', landsat_AOI.size().getInfo())

Total number of images : 230


In [ ]:
def addtemp(image):
  temp = image.select('ST_B.*').multiply(0.00341802).add(149.0).subtract(273.15).rename('temperature')
  return image.addBands(temp)
with_temp = landsat_AOI.map(addtemp)


In [ ]:
def meantemp(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['temperature']),
                           geometry = lendiya_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('temperature').getInfo()

In [ ]:
Images_temp = with_temp.select('temperature').toList(with_temp.size())
temp_coll = []
for i in range(Images_temp.length().getInfo()):
  image = ee.Image(Images_temp.get(i-1))
  tempe_temp = meantemp(image)
  temp_coll.append(tempe_temp)

In [ ]:
dates = np.array(with_temp.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

In [ ]:
df5 = pd.DataFrame(temp_coll, index = day, columns = ['Temperature'])
df5.index = pd.to_datetime(df5.index, format="%Y/%m/%d")
df5.sort_index(ascending = True, inplace = True)

In [ ]:
df5

,Temperature
2016-01-07,18.151491
2016-01-14,25.201303
2016-01-23,23.758434
2016-01-30,25.482449
2016-02-08,23.369908
...,...
2021-11-27,26.894394
2021-12-06,30.914079
2021-12-13,29.534561
2021-12-22,28.716595


In [ ]:
df5.to_csv('Temperature.csv')

# **Dissolved Oxygen Matter**


Selecting the Satellite

In [ ]:
sentinel3 = ee.ImageCollection("COPERNICUS/S3/OLCI").\
              filterDate("2018-08-01","2018-12-31")

lendiya_AOI = ee.Geometry.Polygon([[77.39023917078465, 23.27477380472318],
          [77.39034645914525, 23.274596401498602],
          [77.39034645914525, 23.273876930443446],
          [77.39060395121068, 23.273847363056678],
          [77.3905717647025, 23.27359111209644],
          [77.3909150874564, 23.273729093443976],
          [77.39105456232518, 23.273847363056678],
          [77.39120476603001, 23.273876930443446],
          [77.39134424089879, 23.27427116163988],
          [77.39111893534154, 23.275808652157437],
          [77.39099018930882, 23.27569038428615],
          [77.39077561258763, 23.27569038428615],
          [77.3904108321616, 23.275394714148526]])
sentinel3_AOI = sentinel3.filterBounds(lendiya_AOI)

In [ ]:
print('Total number of images :', sentinel3_AOI.size().getInfo())

Total number of images : 492


In [ ]:
def addDM(image):
  rgb = image.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance'])\
              .multiply(ee.Image([0.00876539, 0.0123538, 0.0115198]))
  dm = rgb.select('Oa08_radiance').divide(rgb.select('Oa04_radiance')).rename('dom')
  return image.addBands(dm)
with_dm = sentinel3_AOI.map(addDM)

In [ ]:
def meanDM(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['dom']),
                           geometry = lendiya_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('dom').getInfo()

In [ ]:
Images_dm = with_dm.select('dom').toList(with_dm.size())
dm_coll= []
for i in range(Images_dm.length().getInfo()):
  image = ee.Image(Images_dm.get(i-1))
  temp_dm = meanDM(image)
  dm_coll.append(temp_dm)

In [ ]:
dates = np.array(with_dm.aggregate_array("system:time_start").getInfo())
day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

In [ ]:
df6 = pd.DataFrame(dm_coll, index = day, columns = ['Dissolved Oxygen Matter'])
df6.index = pd.to_datetime(df6.index, format="%Y/%m/%d")
df6.sort_index(ascending = True, inplace = True)

In [ ]:
df6

In [ ]:
df6.to_csv("dom\AUG1_2018toDEC31_2018")

# **Suspended Matter**

In [ ]:
def addSM(image):
  rgb = image.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance'])\
              .multiply(ee.Image([0.00876539, 0.0123538, 0.0115198]))
  suspended_matter = rgb.select('Oa08_radiance').divide(rgb.select('Oa06_radiance')).rename('suspended_matter')
  return image.addBands(suspended_matter)
with_sm = sentinel3_AOI.map(addSM)

In [ ]:
def meanSM(image):
  image = ee.Image(image)
  mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['suspended_matter']),
                           geometry = sarapangi_AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
  return mean.get('suspended_matter').getInfo()

In [ ]:
Images_sm = with_sm.select('suspended_matter').toList(with_sm.size())
sm_coll= []
for i in range(Images_sm.length().getInfo()):
  image = ee.Image(Images_sm.get(i-1))
  temp_sm = meanSM(image)
  sm_coll.append(temp_sm)